# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe DataFramesMeta"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-02-05 11:33:54 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing Julia package DataFramesMeta...


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [2]:
using DataFrames
using CSV
using Pipe
using DataFramesMeta
using Dates

# Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv).

In [3]:
URL = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv"
download(URL, "US_Crime_Rates_1960_2014.csv")

"US_Crime_Rates_1960_2014.csv"

# Step 3. Assign it to a variable called crime.

In [4]:
crime = CSV.read("US_Crime_Rates_1960_2014.csv", DataFrame)

,Year,Population,Total,Violent,Property,Murder,Forcible_Rape,Robbery
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1960,179323175,3384200,288460,3095700,9110,17190,107840
2,1961,182992000,3488000,289390,3198600,8740,17220,106670
3,1962,185771000,3752200,301510,3450700,8530,17550,110860
4,1963,188483000,4109500,316970,3792500,8640,17650,116470
5,1964,191141000,4564600,364220,4200400,9360,21420,130390
6,1965,193526000,4739400,387390,4352000,9960,23410,138690
7,1966,195576000,5223500,430180,4793300,11040,25820,157990
8,1967,197457000,5903400,499930,5403500,12240,27620,202910
9,1968,199399000,6720200,595010,6125200,13800,31670,262840


# Step 4. What is the type of the columns?

In [5]:
eltype.(eachcol(crime))

12-element Vector{DataType}:
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64

# Step 5. Convert the type of the column Year to datetime64

In [6]:
crime.Year = Date.(crime.Year)

55-element Vector{Date}:
 1960-01-01
 1961-01-01
 1962-01-01
 1963-01-01
 1964-01-01
 1965-01-01
 1966-01-01
 1967-01-01
 1968-01-01
 1969-01-01
 1970-01-01
 1971-01-01
 1972-01-01
 ⋮
 2003-01-01
 2004-01-01
 2005-01-01
 2006-01-01
 2007-01-01
 2008-01-01
 2009-01-01
 2010-01-01
 2011-01-01
 2012-01-01
 2013-01-01
 2014-01-01

In [10]:
eltype.(eachcol(crime))

11-element Vector{DataType}:
 Date
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64
 Int64

# Step 6. Set the Year column as the index of the dataframe

In [7]:
# This doesn't have much meaning in DataFrames.jl

# Step 7. Delete the Total column

In [8]:
select!(crime, Not(:Total)) # This is a supremely weird way to delte columns

,Year,Population,Violent,Property,Murder,Forcible_Rape,Robbery,Aggravated_assault
,Date,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1960-01-01,179323175,288460,3095700,9110,17190,107840,154320
2,1961-01-01,182992000,289390,3198600,8740,17220,106670,156760
3,1962-01-01,185771000,301510,3450700,8530,17550,110860,164570
4,1963-01-01,188483000,316970,3792500,8640,17650,116470,174210
5,1964-01-01,191141000,364220,4200400,9360,21420,130390,203050
6,1965-01-01,193526000,387390,4352000,9960,23410,138690,215330
7,1966-01-01,195576000,430180,4793300,11040,25820,157990,235330
8,1967-01-01,197457000,499930,5403500,12240,27620,202910,257160
9,1968-01-01,199399000,595010,6125200,13800,31670,262840,286700


# Step 8. Group the year by decades and sum the values

In [ ]:
# Honestly can think of any good approach at the moment. will add something better later